# 18. Reinforcement Learning

### Learning to Optimize Rewards

In Reinforcement Learning, a software **agent** makes **observations** and takes **actions** within an **environment**, and in return it receives **rewards**. 

### Policy Search

The algorithm a software agent uses to determine its actions is called its **policy**. The crux of the matter is: how do we find the best (e.g. least time / energy consuming, etc.) policy? This is what **policy search** is all about. There are different approaches:

1. **Brute force**: Try out many different values for the parameters that define our actions, and pick the combination that performs best.   

2. **Genetic algorithms**: Randomly create N policies and try them out, then kill worst X% and make more policies (e.g. adding random variation) out of the remaining ones. 

3. **Policy gradients**: Evaluating the gradients of the rewards with regard to the policy parameters, then tweaking these parameters by following the gradients toward higher rewards. 

### Introduction to OpenAI Gym

One of the challenges of Reinforcement Learning is that in order to train an agent, you first need to have a working environment. Training in the real world is hard and expensive, so we resort to a simulated environment. **OpenAI Gym** provides such an environment. 

In [1]:
import gym

In [2]:
env = gym.make("CartPole-v1")

In [3]:
obs = env.reset()

In [4]:
obs

array([ 0.03800524,  0.0479583 , -0.04479541,  0.04167823])

This is a 2D simulation of a cart that can be accelerated left or right in order to balance a pole placed on top of it. 

* Horizontal position (0.0 = center)
* Velocity (positive = right)
* Angle of the pole (0.0 = vertical)
* Angular velocity (positive = clockwise)

Let's see which actions are possible in this env:

In [5]:
env.action_space

Discrete(2)

Two possible dicrete values are allowed (accelerating left = 0 or right = 1). Since our pole is leaning right we will move right: 

In [6]:
action = 1 # accelerate right

In [7]:
obs, reward, done, info = env.step(action) # excecute new action

In [8]:
obs

array([ 0.03896441,  0.24369302, -0.04396185, -0.26479478])

In [9]:
reward # in this env reward is always 1

1.0

In [10]:
done # True when episode is over

False

In [11]:
info 

{}

Let's hardcode a policy: accelerate left when the pole is leaning toward the left and accelerates right when the pole is leaning toward the right.

In [12]:
def basic_policy(obs):
    angle = obs[2]
    return 0 if angle < 0 else 1

totals = [] # rewards over 500 episodes
for episode in range(500):
    episode_rewards = 0
    obs = env.reset()
    for step in range(200):
        action = basic_policy(obs)
        obs, reward, done, info = env.step(action)
        episode_rewards += reward
        if done:
            break
    totals.append(episode_rewards)

In [13]:
# results
import numpy as np
np.mean(totals), np.std(totals), np.min(totals), np.max(totals)

(41.878, 8.707187605650862, 24.0, 66.0)

### Neural Network Policies

Our NN will estimate a probability for each action, and then we will select an action randomly, according to the estimated probabilities. 

In [14]:
import tensorflow as tf
from tensorflow import keras

n_inputs = env.observation_space.shape[0] # = 4
model = keras.models.Sequential([
keras.layers.Dense(5, activation="elu", input_shape=[n_inputs]),
keras.layers.Dense(1, activation="sigmoid"),
])

How do we train it? 

### Evaluating Actions: The Credit Assignment Problem

It's not possible to use our usual supervised approach here. For example, if the agent manages to balance the pole for 100 steps, how can it know which of the 100 actions it took were good, and which of them were bad? In other words, there is no target probability distribution to learn from. 

A strategy to tackle this issue is to evaluate an action based on the sum of all the rewards that come after it, usually applying a discount factor $\gamma$ at each step. 

Return = Reward 1 + ($\gamma$ x Reward 2) + ($\gamma^2$ x Reward 3)

The higher $\gamma$ the more future rewards will count as much as present ones. 

### Policy Gradients

One popular class of PG algorithms is called REINFORCE algorithms:

1. First, let the neural network policy play the game several times, and at each step, compute the gradients that would make the chosen action even more likely—but don’t apply these gradients yet

2. Once you have run several episodes, compute each action’s advantage (how much better or worse an action is, compared to the other possible actions)

3. If an action’s advantage is positive, it means that the action was probably good, and you want to apply the gradients computed earlier to make the action even more likely to be chosen in the future

4. Compute the mean of all the resulting gradient vectors, and use it to perform a Gradient Descent step

Implementation for one step:

In [15]:
def play_one_step(env, obs, model, loss_fn):
    with tf.GradientTape() as tape:
        left_proba = model(obs[np.newaxis])
        action = (tf.random.uniform([1, 1]) > left_proba)
        y_target = tf.constant([[1.]]) - tf.cast(action, tf.float32)
        loss = tf.reduce_mean(loss_fn(y_target, left_proba))
    grads = tape.gradient(loss, model.trainable_variables)
    obs, reward, done, info = env.step(int(action[0, 0].numpy()))
    return obs, reward, done, grads

Now let’s create another function that will rely on the `play_one_step()` function to play multiple episodes, returning all the rewards and gradients for each episode and each step:

In [16]:
def play_multiple_episodes(env, n_episodes, n_max_steps, model,
    loss_fn):
        all_rewards = []
        all_grads = []
        for episode in range(n_episodes):
            current_rewards = []
            current_grads = []
            obs = env.reset()
            for step in range(n_max_steps):
                obs, reward, done, grads = play_one_step(env, obs, model, loss_fn)
                current_rewards.append(reward)
                current_grads.append(grads)
                if done:
                    break
            all_rewards.append(current_rewards)
            all_grads.append(current_grads)
        return all_rewards, all_grads

Now we need two more functions: the first to compute the sum of future discounted rewards at each step, and the second will normalize all these discounted rewards (returns).

In [17]:
def discount_rewards(rewards, discount_factor):
    discounted = np.array(rewards)
    for step in range(len(rewards) - 2, -1, -1):
        discounted[step] += discounted[step + 1] * discount_factor
    return discounted

# normalized action advantages
def discount_and_normalize_rewards(all_rewards, discount_factor):
    all_discounted_rewards = [discount_rewards(rewards, discount_factor)
                              for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(discounted_rewards - reward_mean) / reward_std
           for discounted_rewards in all_discounted_rewards]

Time to define the hyperparams:

In [18]:
n_iterations = 150
n_episodes_per_update = 10
n_max_steps = 200
discount_factor = 0.95

And optimizer and loss function:

In [19]:
optimizer = keras.optimizers.Adam(lr=0.01)
loss_fn = keras.losses.binary_crossentropy

Ready for the training loop:

In [20]:
for iteration in range(n_iterations):
    all_rewards, all_grads = play_multiple_episodes(
        env, n_episodes_per_update, n_max_steps, model, loss_fn)
    all_final_rewards = discount_and_normalize_rewards(all_rewards,
                                                        discount_factor)
    all_mean_grads = []
    for var_index in range(len(model.trainable_variables)):
        mean_grads = tf.reduce_mean(
            [final_reward * all_grads[episode_index][step][var_index]
            for episode_index, final_rewards in enumerate(all_final_rewards)
                 for step, final_reward in enumerate(final_rewards)], axis=0)
        all_mean_grads.append(mean_grads)
    optimizer.apply_gradients(zip(all_mean_grads,
model.trainable_variables))

And we are done! 

### Markov Decision Processes

In the early 20th century, the mathematician Andrey Markov studied **stochastic processes with no memory**, called **Markov chains**. 
* Fixed number of states, and randomly evolves from one state to another
* The probability for it to evolve from a state `s` to a state `s′` is fixed
* The probability only depends on `(s, s')` (no memory)

Markov decision processes were introduced in the 1950s: at each step, an agent can choose one of several possible actions, and the transition probabilities depend on the chosen action.  

Moreover, some state transitions return some reward (positive or negative), and the agent’s goal is to find a policy that will maximize reward over time.

Bellman found a way to estimate the optimal state $V*(s)$ value of any state $s$ as the sum of all discounted future rewards the agent can expect on average after it reaches a state, assuming it acts optimally. 

In this case, the optimal state is given by:

$V^*(s) = max_a \sum_s T(s,a,s')[R(s,a,s') + \gamma \cdot V^*(s')] \text{for all s}$ 

$\gamma$ = discount factor 

Bellman found a very similar algorithm to estimate the optimal state-action values, generally called **Q-Values** (Quality Values). The optimal Q-Value of the state-action pair $(s, a)$ is the sum of discounted future rewards the agent can expect on average after it reaches the state s and chooses action a, but before it sees the outcome of this action, assuming it acts optimally after that action.

$Q_{k+1}(s,a) \leftarrow \sum_s' T(s,a,s') [R(s,a,s') + \gamma \cdot max_a'  Q_k(s',a')] \text{for all} (s'a)$

Once we have optimal Q-Values, the optimal policy is the one that chooses the action that maximizes for every state.  

Implementation:

In [21]:
transition_probabilities = [ # shape=[s, a, s']
[[0.7, 0.3, 0.0], [1.0, 0.0, 0.0], [0.8, 0.2, 0.0]],
[[0.0, 1.0, 0.0], None, [0.0, 0.0, 1.0]],
[None, [0.8, 0.1, 0.1], None]]

rewards = [ # shape=[s, a, s']
[[+10, 0, 0], [0, 0, 0], [0, 0, 0]],
[[0, 0, 0], [0, 0, 0], [0, 0, -50]],
[[0, 0, 0], [+40, 0, 0], [0, 0, 0]]]

possible_actions = [[0, 1, 2], [0, 2], [1]]

Notation = `[first state][action][second state]`

In [22]:
Q_values = np.full((3, 3), -np.inf) # -np.inf for impossible actions
for state, actions in enumerate(possible_actions):
    Q_values[state, actions] = 0.0 # initialization

In [23]:
# Q-Value Iteration algorithm

gamma = 0.90 # discount factor

for iteration in range(50):
    Q_prev = Q_values.copy()
    for s in range(3):
        for a in possible_actions[s]:
            Q_values[s, a] = np.sum([
                transition_probabilities[s][a][sp]
                * (rewards[s][a][sp] + gamma * np.max(Q_prev[sp]))
            for sp in range(3)])

Let's look at the results:

In [24]:
Q_values

array([[18.91891892, 17.02702702, 13.62162162],
       [ 0.        ,        -inf, -4.87971488],
       [       -inf, 50.13365013,        -inf]])

### Temporal Difference Learning

The Temporal Difference Learning (TD Learning) algorithm is tweaked to take into account the fact that the agent has only partial knowledge of the Markov Decision Process. 

### Q-Learning

Q-Learning algorithm is an adaptation of the Q-Value Iteration algorithm to the situation where the transition probabilities and the rewards are initially unknown.

$Q(s,a) \leftarrow r + \gamma \cdot max_a' Q(s',a') $

In [25]:
# random exploration

def step(state, action):
    probas = transition_probabilities[state][action]
    next_state = np.random.choice([0, 1, 2], p=probas)
    reward = rewards[state][action][next_state]
    return next_state, reward

In [26]:
def exploration_policy(state):
    return np.random.choice(possible_actions[state])

In [27]:
alpha0 = 0.05 # initial learning rate
decay = 0.005 # learning rate decay
gamma = 0.90 # discount factor
state = 0 # initial state

for iteration in range(10000):
    action = exploration_policy(state)
    next_state, reward = step(state, action)
    next_value = np.max(Q_values[next_state])
    alpha = alpha0 / (1 + iteration * decay)
    Q_values[state, action] *= 1 - alpha
    Q_values[state, action] += alpha * (reward + gamma * next_value)
    state = next_state

#### Exploration Policies

Let's improve our exploration policies. A better one than the one (random) we used before is $\epsilon$-greedy policy. At each step is random with probability $\epsilon$ or greedy (highest Q-value) with prob 1-$\epsilon$. Usually $\epsilon$ will start at 1 and decrease over time. 

#### Approximate Q-Learning and Deep Q-Learning

The problem with Q-learning is that it doesn't scale particularly well. The solution is to find a function $Q_\theta (s,a)$ that approximates the Q-Value of any state-action pair $(s, a)$ using a manageable number of parameters.

Initially, linear combinations were used. Then, Deepmind came along and used DNN to estimate Q-values (Deep Q-Network). To estimate this sum of future discounted rewards, we can simply execute the DQN on the next state s′ and for all possible actions a′.

### Implementing Deep Q-Learning

To solve the CartPole environment, we do not need a very complicated neural net; a couple of hidden layers will do:

In [28]:
env = gym.make("CartPole-v0")
input_shape = [4] # == env.observation_space.shape
n_outputs = 2 # == env.action_space.n

model = keras.models.Sequential([
    keras.layers.Dense(32, activation="elu", input_shape=input_shape),
    keras.layers.Dense(32, activation="elu"),
    keras.layers.Dense(n_outputs)
])

In [29]:
def epsilon_greedy_policy(state, epsilon=0):
    if np.random.rand() < epsilon:
        return np.random.randint(2)
    else:
        Q_values = model.predict(state[np.newaxis])
        return np.argmax(Q_values[0])

Instead of training the DQN based only on the latest experiences, we will store all experiences in a replay buffer (or replay memory), and we will sample a random training batch from it at each training iteration.

In [30]:
from collections import deque

replay_buffer = deque(maxlen=2000)

In [31]:
# sample a random batch of experiences

def sample_experiences(batch_size):
    indices = np.random.randint(len(replay_buffer), size=batch_size)
    batch = [replay_buffer[index] for index in indices]
    states, actions, rewards, next_states, dones = [
        np.array([experience[field_index] for experience in batch])
        for field_index in range(5)]
    return states, actions, rewards, next_states, dones

In [32]:
# play a single step using e-greedy policy
# and store experience in replay buffer

def play_one_step(env, state, epsilon):
    action = epsilon_greedy_policy(state, epsilon)
    next_state, reward, done, info = env.step(action)
    replay_buffer.append((state, action, reward, next_state, done))
    return next_state, reward, done, info

Finally, sample a batch of experiences from the replay buffer and train the DQN by performing a single Gradient Descent step:

In [33]:
batch_size = 32
discount_factor = 0.95
optimizer = keras.optimizers.Adam(lr=1e-3)
loss_fn = keras.losses.mean_squared_error

def training_step(batch_size):
    experiences = sample_experiences(batch_size)
    states, actions, rewards, next_states, dones = experiences
    next_Q_values = model.predict(next_states)
    max_next_Q_values = np.max(next_Q_values, axis=1)
    target_Q_values = (rewards +
                      (1 - dones) * discount_factor * max_next_Q_values)
    mask = tf.one_hot(actions, n_outputs)
    with tf.GradientTape() as tape:
        all_Q_values = model(states)
        Q_values = tf.reduce_sum(all_Q_values * mask, axis=1,
    keepdims=True)
        loss = tf.reduce_mean(loss_fn(target_Q_values, Q_values))
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

Training the model:

In [40]:
for episode in range(600):
    obs = env.reset()
    for step in range(200):
        epsilon = max(1 - episode / 500, 0.01)
        obs, reward, done, info = play_one_step(env, obs, epsilon)
    if done:
        break
if episode > 50:
    training_step(batch_size)

C:\Users\giuse\Anaconda3\envs\py_ml\lib\site-packages\gym\logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
